In [ ]:
import bs4 # TODO: Achar o motto do bs4
from bs4.element import NavigableString
import re, logging
import dateutil.parser
import pandas as pd
from collections import namedtuple
import settings

logging.basicConfig(level=logging.WARNING)
log = logging.getLogger()

In [ ]:
Msg = namedtuple('Msg', 'text name datetime')
SubjectChange = namedtuple('SubjectChange', 'subject phone datetime')

In [ ]:
with open(f'{settings.GROUP_NAME}.html') as f:
    html = f.read()
    soup = bs4.BeautifulSoup(html, 'html5lib')

In [ ]:
def get_messages(soup): # TODO: Emojis, Images, Quotes, Audios
    msgs, subject_changes = [], []
    date, phone = None, None
    body = soup.find('body')
    for div in body.select('[data-pre-plain-text]'):
        log.debug('------------------')
        log.debug(div)
        datetime_author = div.attrs['data-pre-plain-text']
        match = re.search(r'\[(.*)\] (.*): ', div.attrs['data-pre-plain-text'])
        datetime, author = match.groups()
        datetime = dateutil.parser.parse(datetime)
        text = div.text
        msg = Msg(text, author, datetime)
        log.info(f'Got a msg {msg}')
        msgs.append(msg)  
    return msgs


messages = get_messages(soup)
messages[:20] # TODO: Resolver quotes q estao vazando no texto dos outros

In [ ]:
df = pd.DataFrame(messages)
df.to_csv(f'{settings.GROUP_NAME}_messages.csv', index=False)